Target : Model skeleton which learns with less parameters.

Result : Params:27,006
         Best Training ACC:9.8%
         Best Test ACC:9.7%
Analysis : Add Batch Normalisation after each convolution  layer.

In [1]:
!pip install torchsummary

In [2]:
from __future__ import print_function
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torchsummary import summary

In [17]:

class Net(nn.Module):
    def __init__(self):
        super(Net, self).__init__()
        self.block1 = nn.Sequential(nn.Conv2d(1, 8, 3, padding=1),
                                    nn.ReLU(),
                                    nn.Conv2d(8, 12, 3, padding=1),
                                    nn.ReLU())
        self.block2 = nn.Sequential(nn.Conv2d(12, 16, 3, padding=1),
                                    nn.ReLU(),
                                    nn.Conv2d(16, 20, 3, padding=1),
                                    nn.ReLU())
        self.block3 = nn.Sequential(nn.Conv2d(20, 24, 3, padding=1),
                                    nn.ReLU(),
                                    nn.Conv2d(24, 28, 3, padding=1),
                                    nn.ReLU())
        self.block4 = nn.Sequential(nn.Conv2d(28, 32, 3, padding=1),
                            nn.ReLU(),
                            nn.Conv2d(32, 10, 3, padding=1),
                            nn.ReLU())
        # self.conv1 = nn.Conv2d(1, 16, 3, padding=1) #input -? OUtput? RF
        # self.conv2 = nn.Conv2d(16, 16, 3, padding=1)
        # self.pool1 = nn.MaxPool2d(2, 2)

        # self.conv3 = nn.Conv2d(16, 32, 3, padding=1) #input -? OUtput? RF
        # self.conv4 = nn.Conv2d(32, 10, 3, padding=1)
        # self.pool2 = nn.MaxPool2d(2, 2)

        # self.conv5 = nn.Conv2d(16, 32, 3, padding=1) #input -? OUtput? RF
        # self.conv6 = nn.Conv2d(32, 32, 3, padding=1)
        # self.conv7 = nn.Conv2d(32, 10, 3, padding=1)
    def forward(self, x):
        x = self.block1(x)
        x = self.block2(x)
        x = self.block3(x)
        x = self.block4(x)
        x = F.avg_pool2d(x,x.size()[2:])
        # print(x.shape)
        x = x.reshape(-1,10)
        
        return x#F.log_softmax(x)

In [11]:
# class Net(nn.Module):
#     def __init__(self):
#         super(Net, self).__init__()
#         # Input Block
#         self.convblock1 = nn.Sequential(
#             nn.Conv2d(in_channels=1, out_channels=10, kernel_size=(3, 3), padding=0, bias=False),
            
#             nn.ReLU()
#         ) # output_size = 26

#         # CONVOLUTION BLOCK 1
#         self.convblock2 = nn.Sequential(
#             nn.Conv2d(in_channels=10, out_channels=10, kernel_size=(3, 3), padding=0, bias=False),
            
#             nn.ReLU()
#         ) # output_size = 24
#         self.convblock3 = nn.Sequential(
#             nn.Conv2d(in_channels=10, out_channels=20, kernel_size=(3, 3), padding=0, bias=False),
            
            
#             nn.ReLU()
#         ) # output_size = 22

#         # TRANSITION BLOCK 1
#         self.pool1 = nn.MaxPool2d(2, 2) # output_size = 11
#         self.convblock4 = nn.Sequential(
#             nn.Conv2d(in_channels=20, out_channels=10, kernel_size=(1, 1), padding=0, bias=False),
            
            
#             nn.ReLU()
#         ) # output_size = 11

#         # CONVOLUTION BLOCK 2
#         self.convblock5 = nn.Sequential(
#             nn.Conv2d(in_channels=10, out_channels=10, kernel_size=(3, 3), padding=0, bias=False),
            
            
#             nn.ReLU()
#         ) # output_size = 9
#         self.convblock6 = nn.Sequential(
#             nn.Conv2d(in_channels=10, out_channels=20, kernel_size=(3, 3), padding=0, bias=False),
#             nn.ReLU()
#         ) # output_size = 7
#         self.pool2 = nn.MaxPool2d(2, 2) 
#         # OUTPUT BLOCK
#         self.convblock7 = nn.Sequential(
#             nn.Conv2d(in_channels=20, out_channels=10, kernel_size=(1, 1), padding=0, bias=False),
#             nn.ReLU()
#         ) # output_size = 7
#         self.convblock8 = nn.Sequential(
#             nn.Conv2d(in_channels=10, out_channels=10, kernel_size=(7, 7), padding=0, bias=False),
#             # nn.BatchNorm2d(10), NEVER
#             # nn.ReLU() NEVER!
#         ) # output_size = 1

#     def forward(self, x):
#         x = self.convblock1(x)
#         x = self.convblock2(x)
#         x = self.convblock3(x)
#         x = self.pool1(x)
#         x = self.convblock4(x)
#         x = self.convblock5(x)
#         x = self.convblock6(x)
#         x = self.pool2(x)
#         x = self.convblock7(x)
#         x = self.convblock8(x)
#         x = x.view(-1, 10)
#         return F.log_softmax(x, dim=-1)

In [18]:


use_cuda = torch.cuda.is_available()
device = torch.device("cuda" if use_cuda else "cpu")
model = Net().to(device)
summary(model, input_size=(1, 28, 28))

----------------------------------------------------------------
        Layer (type)               Output Shape         Param #
            Conv2d-1            [-1, 8, 28, 28]              80
              ReLU-2            [-1, 8, 28, 28]               0
            Conv2d-3           [-1, 12, 28, 28]             876
              ReLU-4           [-1, 12, 28, 28]               0
            Conv2d-5           [-1, 16, 28, 28]           1,744
              ReLU-6           [-1, 16, 28, 28]               0
            Conv2d-7           [-1, 20, 28, 28]           2,900
              ReLU-8           [-1, 20, 28, 28]               0
            Conv2d-9           [-1, 24, 28, 28]           4,344
             ReLU-10           [-1, 24, 28, 28]               0
           Conv2d-11           [-1, 28, 28, 28]           6,076
             ReLU-12           [-1, 28, 28, 28]               0
           Conv2d-13           [-1, 32, 28, 28]           8,096
             ReLU-14           [-1, 32,

In [19]:


torch.manual_seed(1)
batch_size = 128

kwargs = {'num_workers': 1, 'pin_memory': True} if use_cuda else {}
train_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=True, download=True,
                    transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)
test_loader = torch.utils.data.DataLoader(
    datasets.MNIST('../data', train=False, transform=transforms.Compose([
                        transforms.ToTensor(),
                        transforms.Normalize((0.1307,), (0.3081,))
                    ])),
    batch_size=batch_size, shuffle=True, **kwargs)


In [20]:
def train(model, device, train_loader, optimizer, epoch):
    average_epoch_loss = 0
    correct_predictions_epoch = 0
    model.train()
    with tqdm(train_loader, unit="batch") as tepoch:
        for batch_idx, (data, target) in enumerate(tepoch):
            tepoch.set_description(f"Train Epoch {epoch}")
            data, target = data.to(device), target.to(device)
            optimizer.zero_grad()
            output = model(data)
            loss = F.nll_loss(output, target)
            loss.backward()
            optimizer.step()
            average_epoch_loss += loss.item()
            correct_predictions = sum(output.argmax(dim = 1) == target)
            correct_predictions_epoch += correct_predictions
            tepoch.set_postfix(loss = round(loss.item(),5), accuracy = format(100 *correct_predictions.item()/data.shape[0], '.4f'))
    return average_epoch_loss/batch_idx,correct_predictions_epoch/len(train_loader.dataset)

In [21]:
from tqdm.notebook import tqdm

In [22]:
def test(model, device, test_loader):
    average_epoch_loss = 0
    correct_predictions_epoch = 0
    model.eval()
    with torch.no_grad():
        with tqdm(test_loader, unit="batch") as tepoch:
            for batch_idx, (data, target) in enumerate(tepoch):
                tepoch.set_description(f"Test  Epoch {epoch}")
                data, target = data.to(device), target.to(device)
                output = model(data)
                loss = F.nll_loss(output, target)  # sum up batch loss
                average_epoch_loss += loss.item()
                correct_predictions = sum(output.argmax(dim = 1) == target)  # get the index of the max log-probability
                correct_predictions_epoch += correct_predictions
                tepoch.set_postfix(loss = round(loss.item(),5), accuracy = format(100 *correct_predictions.item()/data.shape[0], '.4f'))
    return average_epoch_loss/batch_idx,correct_predictions_epoch/len(test_loader.dataset)

In [24]:

model = Net().to(device)
optimizer = optim.SGD(model.parameters(), lr=0.01, momentum=0.9)
train_loss = []
test_loss = []

train_accuracy = []
test_accuracy = []
for epoch in range(1, 19):

    tr_loss,tr_acc = train(model, device, train_loader, optimizer, epoch)
    tst_loss,tst_acc = test(model, device, test_loader)
    train_loss.append(tr_loss),train_accuracy.append(tr_acc)
    test_loss.append(tst_loss),test_accuracy.append(tst_acc)
    print("Train_epoch : ",tr_acc.item())
    print("Test_epoch : ",tst_acc.item())



Train_epoch :  0.0994499996304512
Test_epoch :  0.09799999743700027



Train_epoch :  0.09871666878461838
Test_epoch :  0.09799999743700027



Train_epoch :  0.09871666878461838
Test_epoch :  0.09799999743700027



Train_epoch :  0.09871666878461838
Test_epoch :  0.09799999743700027



Train_epoch :  0.09871666878461838
Test_epoch :  0.09799999743700027


...


Train_epoch :  0.09871666878461838
Test_epoch :  0.09799999743700027



Train_epoch :  0.09871666878461838
Test_epoch :  0.09799999743700027



Train_epoch :  0.09871666878461838
Test_epoch :  0.09799999743700027



Train_epoch :  0.09871666878461838
Test_epoch :  0.09799999743700027



Train_epoch :  0.09871666878461838
Test_epoch :  0.09799999743700027
